In [1]:
import IPython
import reboting
from reboting import CouldntSaveDataException
from reboting import CsvDownloadAndParsingException
from reboting import CouldNotDownloadFileException
from reboting import CouldntCreateVisualException
from reboting import DataTooBigException
from reboting import NoVisualsPossible
import requests
import pandas as pd

go over all queries for each bundesland

- for every item ask the bot with url
- and then analyze the db

In [2]:
stats = {}
config = {
    'wien' : '&l=http://sws.geonames.org/2761369/&l=http://sws.geonames.org/2761367/&l=http://sws.geonames.org/2761333/&l=http://sws.geonames.org/2761368/'
}
for key, value in config.items():
    print(key+" val: "+value)
    offset=0
    url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
    r = requests.get(url)
    resp = r.json()  
    stats[key]={ 'total': resp['total'], 'success' : 0}
    #lets start the for loop again
    for i in range(0,resp['total']):        
        offset=i
        url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
        r = requests.get(url)
        resp = r.json()  
        #hardcoded userid
        user_id='10213970072671093'
        searchresult = resp['results'][0]
        data_desc = {
                "name" : "No name available",
                "description" : "No description available",
                "publisher" : "no publisher available",
                "portal" : "No Portal available",
                "url": searchresult['url'],
                "user_id" : user_id
        }
        print(str(i)+" "+searchresult['url'])
        if searchresult['dataset'] is not None:
            if searchresult['dataset']['dataset_name'] is not None:
                data_desc['name'] = searchresult['dataset']['dataset_name']
            if searchresult['dataset']['dataset_description'] is not None:
                data_desc['description'] = searchresult['dataset']['dataset_description']
        if searchresult['portal'] is not None:
            data_desc['portal'] = searchresult['portal']
        if searchresult['dataset']['publisher'] is not None:
            data_desc['publisher'] = searchresult['dataset']['publisher']
        try:
            slug = reboting.checkforknowncsv(data_desc = data_desc)
            url = 'https://doh.23degrees.io/viz/'+slug
            print(url)
            stats[key]['success'] = stats[key]['success']+1 
        except CouldntSaveDataException as e:
            print("ERROR: couldnt save data on 23 degree server")
        except CsvDownloadAndParsingException as e:
            print("ERROR: something went wrong when downloading the file")
        except CouldNotDownloadFileException as e:
            print("ERROR: couldnt download file")
        except CouldntCreateVisualException as e:
            print("ERROR: couldnt create visual")
        except DataTooBigException as e:
            print("ERROR: too many lines: "+str(e))   
        except NoVisualsPossible as e:
            print("ERROR: no visual possible: "+str(e))   

wien val: &l=http://sws.geonames.org/2761369/&l=http://sws.geonames.org/2761367/&l=http://sws.geonames.org/2761333/&l=http://sws.geonames.org/2761368/
0 https://www.wien.gv.at/statistik/ogd/vie_401.csv
filename: 7587722199975748651.csv.tmp
G90101
5aa01961a1957a0018005027
5aa01961a4e2bd006b0f9f0b
registerzaehlung-2011-zaehlbezirke-1-wien-u_total-6
https://doh.23degrees.io/viz/registerzaehlung-2011-zaehlbezirke-1-wien-u_total-6
1 https://www.wien.gv.at/statistik/ogd/vie_212.csv
filename: 1983807193722578441.csv.tmp
G90101


KeyboardInterrupt: 

In [ ]:
stats